<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 21.1 MB/s 


In [2]:
# Necessary libaries
import pandas as pd
import os
#import PyPDF2

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [3]:
# Import data
path_iss_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')

# lets see frist 3 and last 3 rows od the dataset
data_iss.head(3).append(data_iss.tail(5))

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
112761,PINOTEX SUPERDEC SOR,61009053404,6100,"-0,002"
112762,QUICK GEVINSTER,214196,926,"-0,002"
112763,UKRUDTSDUG BASIC 1X,42139019916,4213,"-0,002"
112764,TRAPPEVANGE,5708832546219,600,"-0,004"
112765,BR MÆRKER RETUR,5766183013538,21,"-0,005"


In [4]:
# Only print one selective row of interest to deep deeper
#data_iss[data_iss.itemname == "BANANER, LØSE" or "ØKO BANANER OR LØSE" or "BANANER"] # or "ØKO BANANER, LØSE", HAVFRISK LAK, APPELSINER 2 KG,Agurk

# only some of the rows of different kinds of the banana
data_iss.iloc[[11, 12, 14, 18], :]   

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


***In above ddataset, only one food itemname (fx `"ØKO BANANER, LØSE"`) is appear in different rows with;***
*   Many different itemnumber, even some are 13 digits and other are 8 digits
*   NULL percent purchage

***Here i like to use NLP to find split itemname to name and other entity***

In [5]:
#!pip install nltk

In [35]:
# Liaries for NLP
from nltk.tokenize import word_tokenize

import nltk
nltk.download("popular")
nltk.download("stopwords")
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("omw-1.4")

stop_words= set(stopwords.words("danish"))
print(stop_words)

# for regular expression
import re

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

{'han', 'mig', 'når', 'min', 'dig', 'under', 'været', 'hendes', 'hende', 'også', 'blive', 'sine', 'var', 'have', 'om', 'dem', 'men', 'thi', 'skal', 'bliver', 'over', 'en', 'til', 'fra', 'et', 'det', 'mange', 'blev', 'vor', 'kunne', 'hvis', 'da', 'nogle', 'nu', 'ikke', 'og', 'deres', 'mit', 'for', 'sin', 'disse', 'ind', 'hvor', 'af', 'sit', 'efter', 'alt', 'denne', 'op', 'er', 'hos', 'havde', 'skulle', 'hans', 'os', 'som', 'din', 'man', 'har', 'jeg', 'jo', 'anden', 'i', 'være', 'den', 'jer', 'sådan', 'mine', 'på', 'meget', 'ad', 'selv', 'du', 'at', 'mod', 'dette', 'sig', 'end', 'ned', 'de', 'her', 'hun', 'ville', 'noget', 'hvad', 'dog', 'eller', 'vi', 'vil', 'ud', 'med', 'ham', 'der', 'alle'}


[nltk_data]    | Downloading package punkt to /root/nltk_data...
[nltk_data]    |   Package punkt is already up-to-date!
[nltk_data]    | Downloading package snowball_data to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection popular
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [36]:
# here i am going to use Regular Express (RegEx) for turning sentance to each word (Word Tokenization)
def remove_delimiters(sentence):
  return re.findall(r'\w+', sentence)

sents= " UKRUDTSDUG BASIC 1X  " # sdujh	ØKO BANANER, LØSE jhghujw, hsdfgw"

trimmed_sent= remove_delimiters(sents)
trimmed_sent

['UKRUDTSDUG', 'BASIC', '1X']

In [37]:
#WWord tagging
words= nltk.word_tokenize(sents)
pos_tag= nltk.pos_tag(words)

words
pos_tag

[('UKRUDTSDUG', 'NNP'), ('BASIC', 'NNP'), ('1X', 'CD')]

In [48]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
# Dependancy parcing
import spacy


import spacy.cli
spacy.cli.download("da_core_news_lg")
nlp= spacy.load('da_core_news_lg')
from spacy import displacy

temp= nlp(sents)
displacy.serve(temp, style= 'dep')

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [58]:
import spacy
from spacy.lang.da.examples import sentences 

nlp = spacy.load("da_core_news_lg")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple overvejer at købe et britisk startup for 1 milliard dollar.
Apple PROPN nsubj
overvejer VERB ROOT
at PART mark
købe VERB obj
et DET det
britisk ADJ amod
startup NOUN obj
for ADP case
1 NUM nummod
milliard NOUN nmod
dollar PROPN nmod
. PUNCT punct


# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [ ]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head(3)

In [ ]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)
# Assign columns name that defined just above
data_frida.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
data_frida.head(3).append(data_frida.tail(3))
data_frida[data_frida.Gruppe == 'Sydfrugt'] #"Frugt"-grøntsager, Sydfrugt, Bærfrugt
#data_frida[data_frida.Navn == "Banan, rå"]


In [ ]:
row=data_frida[data_frida.Navn== "Banan, rå"]   # Kylling, bryst (filet), kogt, pålæg,  ØKO BANANER, LØSE
print("Id: ", row.index[0])
print("Category: ", row.Gruppe.values[0])
print("Name: ", row.Navn.values[0])
print("Total Energy: ", row.Energy_kj.values[0], "Kj (", row.Energy_kcal.values[0], "Kacl)")
print("Total Energy: ", row.Energy_kcal.values[0])
print("Total Energy: ", row.Energy_kj.values[0])

In [ ]:
# To see NAN value and data types of each columns
#data_frida.info()

***Above dataset do not contain NAN value and all of the columns are object type, so some of the columns need to be changed to interger or float data type to work as numbers value***

# Recipt from my personal Storebox account
- This data set i download from my own storebox account. Lets see what kind of information we can extract from storebox account

***The Store-box dataset contain one dataset of recieptand another dataset of the actual list of itemspurchaged each time***

In [ ]:
#Import data
path_storebox_data= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/storbox_recipts.json"

#Loading dat in dataframe 
data_storebox= pd.read_json(path_storebox_data)
# ['id', 'type', 'receiptId', 'purchaseDate', 'purchaseDateTimeString', 'orderNumber', 'price', 'merchant', 'receiptLines', 'terminalId', 'payments', 'cards', 
#     'barcode', 'headerTexts', 'footerTexts', 'userIds', 'imageId', 'receiptImage', 'note', 'tags', 'headerText', 'footerText', 'purchaseDateTimeZoneOffset', 'legalReceipt']

# Lets open only columns that make sence
data_storebox= data_storebox[["id", "receiptId", "purchaseDate", "orderNumber", "price", "merchant",'barcode']]
data_storebox.head(3)

***With the receiptId i can go to indiviisual reciept and extract list of purchage items. However the reciept is in PDF so need little advance RegEx code so i have not done it yet but will update soon.***


**Now lets  open reciept of  of the very first column (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above data**

In [ ]:
# Lets open reciept of very first columns (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above dataset (data_storbox)
data_storbox_reciept= "/content/05jrqpia3ofdzsbq773yme4on0ko1f87-14736470631022876123.pdf" # "https://github.com/mrchapagain/food_ontology/raw/main/storbox_recipt_pdf.pdf"

# read information from PDF file (open it in read binary mode)
pdfFileObj = open(data_storbox_reciept, 'rb')

# call and store PdfFileReader
# object in pdfReader
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# to access first page if the PDF file have multiple pages, pass 0
pageObj = pdfReader.getPage(0)

# extract the page object by extractText() function
texts = pageObj.extractText()

# print the extracted texts
print(texts)

***I have extracted data from PDF file now first need to turn it to dictionary and then DataFrame.***

In [ ]:
lines= texts.rstrip("\n")
lines

# Climate Data from Concito

In [ ]:
#Import data
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [ ]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

# Data from COOP store (store data)

In [ ]:
path_coop_data= "https://raw.githubusercontent.com/mrchapagain/ConsumerDataAnalytics/main/storedata/storedata_csv.csv"
#data_coop= pd.read_excel(path_coop_data, sheet_name=1, header=0, skiprows=0).round(decimals = 2)

data_coop= pd.read_csv(path_coop_data, index_col=0, sep= ";", on_bad_lines='skip', engine='python')

data_coop.head()

***These information are saved wrong way; all the columns of csv in one column so it open like one column***

So i am going to open with string split method with comma " ,"

# TradeSync data from GS1 

In [ ]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [ ]:
#Import data
path_tradesync_data= "/content/testdata_gs1.xml" # "https://github.com/mrchapagain/food_ontology/blob/main/gs1_data/testdata.xml"

In [ ]:
tree = ET.parse(path_tradesync_data)
root = tree.getroot()
print("Roots:", root.tag)
print("Attrib:", root.attrib)
print("Childerns: ", list(root))


***This part is hidden because the datast is very big and  take somany lines***

In [ ]:
# Code to have a look whole data
"""# Reading the data inside the xml file to a variable under the name  data
with open(path_tradesync_data, 'r') as f:
    for line in f.readlines():
      print(line)
"""

# Code to explore tags
"""# The xml file has a LOT of tags!
list_of_items= []
stopAt = 200
for elem in root.iter():
    print(elem.tag)
    list_of_items.append(elem.tag)
    #print(elem.attrib)
    if stopAt==0:
        break
    stopAt -= 1"""

In [ ]:
#print(ET.tostring(root, encoding='utf8').decode('utf8'))
list_of_items_to_find= ["entityIdentification", "brandName", "functionalName", "gln", "gtin", "creationDateTime", "dataRecipient", "contactDescription", "partyAddress", "gpcCategoryCode", "grossWeight", "descriptionShort", "allergenSpecificationName", "ingredientStatement", "tradeItemTemperatureConditionTypeCode", "uniformResourceIdentifier"]
#  "additionalTradeItemIdentification", "partyName",

#Lets start the dictionary to collecct items
dict_item_text = {}

# Loop over the tags that make sense (from list above)
for list_item in list_of_items_to_find:
  
  # Again  looping to get the text value
  for item in root.iter(list_item):
    dict_item_text[list_item] = item.text

# Lets see what we have c collected from the loop
dict_item_text

***Above dictinary are the information extracted from GS1 xml file associated with each item from the purchage list***

*We can grasp a lot of infoation about the item*